In [1]:
!pip install confluent_kafka

In [1]:
import json
import gzip
from confluent_kafka import Producer
import time

In [2]:
def read_reviews(file_path):
    with gzip.open(file_path, 'rb') as file:
        for line_number, line in enumerate(file, start=0):
            # review = json.loads(eval(line))
            # review = eval(line)
            # review = eval(line.replace('null', 'None'))
            review = json.loads(line)
            # review = {k: v if v is not None else None for k, v in review.items()}
            yield review

def kafka_producer(broker, topic, file_path):
    producer_conf = {'bootstrap.servers': broker}
    producer = Producer(producer_conf)
    x = 0

    try:
        for review in read_reviews(file_path):
            x += 1
            
            review['last_update_time'] = time.time()
            # Convert the review to a JSON string
            message_value = json.dumps(review)
            
            # Produce the message to the Kafka topic
            producer.produce(topic, value=message_value.encode('utf-8'))

            # Flush the producer to ensure the message is sent
            producer.flush()

            # Print the data just sent
            print(f"Sent {x} :  {message_value}")

            # Pause for 5 seconds
            time.sleep(1)

    except KeyboardInterrupt:
        pass

    finally:
        # Close the producer (no close method, just flush)
        producer.flush()

In [3]:
if __name__ == '__main__':
    # Thay thế 'localhost:9092' và 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7' bằng thông tin Kafka của bạn
    bootstrap_servers = 'localhost:9092'
    kafka_topic = 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7'
    
    # Thay thế 'data/reviews.clean.json.gz' bằng đường dẫn tới tệp của bạn
    file_path = '../data_vi_tphcm/reviews_HCM_after_31_01_2024.jsonl.gz'
    
    try:
        kafka_producer(bootstrap_servers, kafka_topic, file_path)
    except KeyboardInterrupt:
        print("Stopping the producer...")
        running = False  # Đặt biến cờ thành False để dừng producer

Sent 1 :  {"reviewId": "ChdDSUhNMG9nS0VJQ0FnSUN0cF8zZS13RRAB", "placeId": "ChIJXbidq_sudTERT6eTAa9jY1A", "title": "018 Cafe", "location/lat": 10.7551649, "location/lng": 106.6676778, "categoryName": "Qu\u00e1n c\u00e0 ph\u00ea", "reviewerId": "1.10E+20", "name": "Oh Seatide (seatide)", "stars": 5.0, "text": "\uc634\uce74\ud398\uc785\ub2c8\ub2e4. \uce5c\uc808\ud55c \ubca0\ud2b8\ub0a8 \uc544\uac00\uc528\ub4e4\uacfc \ub300\ud654\ub3c4 \ub098\ub204\uace0 \uc815\ub2f5\uac8c \ub290\uaef4\uc9c0\ub294 \uce74\ud398\uc785\ub2c8\ub2e4", "publishedAtDate": "2024-02-01T09:20:12.196Z", "categories": "Qu\u00e1n c\u00e0 ph\u00ea", "last_update_time": 1707229451.8756864}
Sent 2 :  {"reviewId": "ChZDSUhNMG9nS0VJQ0FnSUR0bklXX1R3EAE", "placeId": "ChIJPZM8uVYpdTERLvi8c7WWMtU", "title": "1 Dessert & Cafe", "location/lat": 10.79644, "location/lng": 106.6857013, "categoryName": "Qu\u00e1n c\u00e0 ph\u00ea", "reviewerId": "1.03E+20", "name": "Oanh Hu\u1ef3nh", "stars": 4.0, "text": "Qu\u00e1n n\u00ean \u0111\u